In [1]:
import datetime
import hashlib
import io
import os
import pathlib
import urllib.request

import pandas as pd

try:
    import xlrd
except ModuleNotFoundError:
    !pip install xlrd
    import xlrd

source = 'https://www.nta.go.jp/publication/pamph/gensen/zeigakuhyo2018/data/01-07.xls'
destination = pathlib.Path(os.environ['WORKSPACE'], 'lib/employment-income/monthly/2019-01-01.csv')

print('At', datetime.datetime.now().astimezone(datetime.timezone.utc).isoformat())
print('Source', source)
print('Destination', destination)

At 2022-06-04T10:29:18.383390+00:00
Source https://www.nta.go.jp/publication/pamph/gensen/zeigakuhyo2018/data/01-07.xls
Destination /home/jovyan/work/lib/employment-income/monthly/2019-01-01.csv


In [2]:
data = io.BytesIO()

print('GET', source)

with urllib.request.urlopen(source) as f:
    data.write(f.read())

print('Hash', hashlib.sha256(data.getvalue()).hexdigest())

GET https://www.nta.go.jp/publication/pamph/gensen/zeigakuhyo2018/data/01-07.xls
Hash bea6690b4c170667d2bd5070a121084d9d620c2bf7605eab806aafb1d996f7ba


In [3]:
data.seek(0)

names = """
不要なカラム
以上
未満
扶養:0
扶養:1
扶養:2
扶養:3
扶養:4
扶養:5
扶養:6
扶養:7
扶養控除なし
""".strip().split()

df = pd.read_excel(data, skiprows=6, names=names, usecols=range(1, 12))

df.head()

,以上,未満,扶養:0,扶養:1,扶養:2,扶養:3,扶養:4,扶養:5,扶養:6,扶養:7,扶養控除なし
0,88000,円未満,0,0,0,0,0,0,0,0,その月の社会保険料等控除後の給与等の金額の3.063％に相当する金額
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88000,89000,130,0,0,0,0,0,0,0,3200
3,89000,90000,180,0,0,0,0,0,0,0,3200
4,90000,91000,230,0,0,0,0,0,0,0,3200


In [4]:
pd.options.display.max_rows = 20
df2 = df.copy()

# 以上 = 未満 - 1 の行を適切に処理
df2['以上'] = df2['以上'].astype(str).str.replace(r'[,円]', r'', regex=True)
df2['以上'] = pd.to_numeric(df2['以上'], errors='coerce')

# 数値に変換できないものを NaN にする
for c in df2.columns:
    df2[c] = pd.to_numeric(df2[c], errors='coerce')

df2 = df2.dropna(how='all')
df2

,以上,未満,扶養:0,扶養:1,扶養:2,扶養:3,扶養:4,扶養:5,扶養:6,扶養:7,扶養控除なし
0,88000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,88000.0,89000.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
3,89000.0,90000.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
4,90000.0,91000.0,230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
5,91000.0,92000.0,290.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
...,...,...,...,...,...,...,...,...,...,...,...
342,857000.0,860000.0,97000.0,89560.0,82130.0,75630.0,69160.0,62700.0,56230.0,49760.0,319300.0
345,860000.0,NaN,97350.0,89920.0,82480.0,75930.0,69470.0,63010.0,56530.0,50070.0,320900.0
358,970000.0,NaN,123190.0,115760.0,108320.0,101770.0,95310.0,88850.0,82370.0,75910.0,NaN
367,1720000.0,NaN,375890.0,368460.0,361020.0,354470.0,348010.0,341550.0,335070.0,328610.0,672200.0


In [5]:
df3 = df2.copy()
# "未満" が null なら "以上" に等しい額のみ該当するようにする
df3.loc[df3['未満'].isnull(), '未満'] = df3['以上'] + 1
df3

,以上,未満,扶養:0,扶養:1,扶養:2,扶養:3,扶養:4,扶養:5,扶養:6,扶養:7,扶養控除なし
0,88000.0,88001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,88000.0,89000.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
3,89000.0,90000.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
4,90000.0,91000.0,230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
5,91000.0,92000.0,290.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
...,...,...,...,...,...,...,...,...,...,...,...
342,857000.0,860000.0,97000.0,89560.0,82130.0,75630.0,69160.0,62700.0,56230.0,49760.0,319300.0
345,860000.0,860001.0,97350.0,89920.0,82480.0,75930.0,69470.0,63010.0,56530.0,50070.0,320900.0
358,970000.0,970001.0,123190.0,115760.0,108320.0,101770.0,95310.0,88850.0,82370.0,75910.0,NaN
367,1720000.0,1720001.0,375890.0,368460.0,361020.0,354470.0,348010.0,341550.0,335070.0,328610.0,672200.0


In [6]:
assert df3.shape == (290, 11)

In [7]:
columns = """
start
end
dependents0
dependents1
dependents2
dependents3
dependents4
dependents5
dependents6
dependents7
unknown
""".strip().split()

for a, b in zip(df3.columns, columns):
    df3.rename(columns={a: c})

df3 = df3.rename(columns={a: b for a, b in zip(df3.columns, columns)})
df3

,start,end,dependents0,dependents1,dependents2,dependents3,dependents4,dependents5,dependents6,dependents7,unknown
0,88000.0,88001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,88000.0,89000.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
3,89000.0,90000.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
4,90000.0,91000.0,230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
5,91000.0,92000.0,290.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0
...,...,...,...,...,...,...,...,...,...,...,...
342,857000.0,860000.0,97000.0,89560.0,82130.0,75630.0,69160.0,62700.0,56230.0,49760.0,319300.0
345,860000.0,860001.0,97350.0,89920.0,82480.0,75930.0,69470.0,63010.0,56530.0,50070.0,320900.0
358,970000.0,970001.0,123190.0,115760.0,108320.0,101770.0,95310.0,88850.0,82370.0,75910.0,NaN
367,1720000.0,1720001.0,375890.0,368460.0,361020.0,354470.0,348010.0,341550.0,335070.0,328610.0,672200.0


In [8]:
df3.to_csv(destination, float_format=int, index=False)